<a href="https://colab.research.google.com/github/Kwasi-Dankwa/farming-yield-project/blob/main/Smart_Farm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model Building with Machine Learning

In [2]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [3]:
# Loading data
df = pd.read_csv("/content/Smart_Farming_Crop_Yield_2024.csv")
df.head()

,farm_id,region,crop_type,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,irrigation_type,...,sowing_date,harvest_date,total_days,yield_kg_per_hectare,sensor_id,timestamp,latitude,longitude,NDVI_index,crop_disease_status
0,FARM0001,North India,Wheat,35.95,5.99,17.79,75.62,77.03,7.27,NaN,...,2024-01-08,2024-05-09,122,4408.07,SENS0001,2024-03-19,14.970941,82.997689,0.63,Mild
1,FARM0002,South USA,Soybean,19.74,7.24,30.18,89.91,61.13,5.67,Sprinkler,...,2024-02-04,2024-05-26,112,5389.98,SENS0002,2024-04-21,16.613022,70.869009,0.58,NaN
2,FARM0003,South USA,Wheat,29.32,7.16,27.37,265.43,68.87,8.23,Drip,...,2024-02-03,2024-06-26,144,2931.16,SENS0003,2024-02-28,19.503156,79.068206,0.80,Mild
3,FARM0004,Central USA,Maize,17.33,6.03,33.73,212.01,70.46,5.03,Sprinkler,...,2024-02-21,2024-07-04,134,4227.80,SENS0004,2024-05-14,31.071298,85.519998,0.44,NaN
4,FARM0005,Central USA,Cotton,19.37,5.92,33.86,269.09,55.73,7.93,NaN,...,2024-02-05,2024-05-20,105,4979.96,SENS0005,2024-04-13,16.568540,81.691720,0.84,Severe


   farm_id             region           crop_type         soil_moisture_.
 Length:500         Length:500         Length:500         Min.   :10.16  
 Class :character   Class :character   Class :character   1st Qu.:17.89  
 Mode  :character   Mode  :character   Mode  :character   Median :25.86  
                                                          Mean   :26.75  
                                                          3rd Qu.:36.02  
                                                          Max.   :44.98  
    soil_pH      temperature_C    rainfall_mm       humidity_.   
 Min.   :5.510   Min.   :15.00   Min.   : 50.17   Min.   :40.23  
 1st Qu.:6.030   1st Qu.:20.30   1st Qu.:119.22   1st Qu.:51.87  
 Median :6.530   Median :24.66   Median :191.54   Median :65.69  
 Mean   :6.524   Mean   :24.68   Mean   :181.69   Mean   :65.19  
 3rd Qu.:7.040   3rd Qu.:29.09   3rd Qu.:239.03   3rd Qu.:78.00  
 Max.   :7.500   Max.   :34.84   Max.   :298.96   Max.   :90.00  
 sunlight_hours   ir

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘RcppEigen’, ‘pbkrtest’, ‘lme4’

trying URL 'https://cran.rstudio.com/src/contrib/RcppEigen_0.3.4.0.2.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/pbkrtest_0.5.4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/lme4_1.1-37.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/car_3.1-3.tar.gz'

The downloaded source packages are in
	‘/tmp/Rtmp0GyXFC/downloaded_packages’
Error in vif(model) : could not find function "vif"


RInterpreterError: Failed to parse and evaluate line '# Load libraries\ninstall.packages("car")\nlibrary(tidyverse)\nmodel <- lm(yield_kg_per_hectare ~ temperature_C + NDVI_index + humidity_. + rainfall_mm + soil_moisture_. + irrigation_type + fertilizer_type, data = df)\nsummary(model)\nvif(model)\n'.
R error message: 'Error in vif(model) : could not find function "vif"'
R stdout:
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘RcppEigen’, ‘pbkrtest’, ‘lme4’

trying URL 'https://cran.rstudio.com/src/contrib/RcppEigen_0.3.4.0.2.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/pbkrtest_0.5.4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/lme4_1.1-37.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/car_3.1-3.tar.gz'

The downloaded source packages are in
	‘/tmp/Rtmp0GyXFC/downloaded_packages’
Error in vif(model) : could not find function "vif"